# Review errors in the results files `metadataset_errors*.csv`

In [1]:
import pandas as pd

errors_df = pd.read_csv("/home/shared/tabzilla/TabSurvey/metadataset_errors.csv")

In [6]:
# filter by experiment
errors_df = errors_df.loc[errors_df["exp_name"].str.contains("algs-gpu-1-datasets-b-v3")]

errors_df.head()

,results_bucket_path,dataset_name,alg_name,hparam_source,trial_number,alg_hparam_id,exp_name,exception
38,results/openml__Census-Income__168340/TabNet/a...,openml__Census-Income__168340,TabNet,default,0,TabNet__seed_0__trial_0,algs-gpu-1-datasets-b-v3_111922_234613_629c.zip,"Traceback (most recent call last):\n File ""/h..."
40,results/openml__Census-Income__168340/TabNet/a...,openml__Census-Income__168340,TabNet,random_1_s0,1,TabNet__seed_0__trial_1,algs-gpu-1-datasets-b-v3_111922_234613_629c.zip,"Traceback (most recent call last):\n File ""/h..."
42,results/openml__Census-Income__168340/TabNet/a...,openml__Census-Income__168340,TabNet,random_2_s0,2,TabNet__seed_0__trial_2,algs-gpu-1-datasets-b-v3_111922_234613_629c.zip,"Traceback (most recent call last):\n File ""/h..."
44,results/openml__Census-Income__168340/TabNet/a...,openml__Census-Income__168340,TabNet,random_3_s0,3,TabNet__seed_0__trial_3,algs-gpu-1-datasets-b-v3_111922_234613_629c.zip,"Traceback (most recent call last):\n File ""/h..."
46,results/openml__Census-Income__168340/TabNet/a...,openml__Census-Income__168340,TabNet,random_4_s0,4,TabNet__seed_0__trial_4,algs-gpu-1-datasets-b-v3_111922_234613_629c.zip,"Traceback (most recent call last):\n File ""/h..."


In [7]:
# how many errors by dataset?
print(f"errors by dataset:\n {errors_df['dataset_name'].value_counts()}")

errors by dataset:
 openml__solar-flare__2068                 30
openml__Census-Income__168340             10
openml__walking-activity__9945             9
openml__ldpa__9974                         6
openml__poker-hand__9890                   6
openml__aloi__12732                        5
openml__chess__3952                        2
openml__kropt__2076                        2
openml__Click_prediction_small__190408     1
Name: dataset_name, dtype: int64


In [10]:
# by alg...
print(f"errors by alg:\n {errors_df['alg_name'].value_counts()}")

errors by alg:
 CatBoost    30
TabNet      21
VIME        20
Name: alg_name, dtype: int64


# Triage errors by alg

In [11]:
errors_by_alg = {}
for alg_name in errors_df["alg_name"].unique():
    errors_by_alg[alg_name] = errors_df.loc[errors_df["alg_name"] == alg_name, "exception"].values

## CatBoost

In [12]:
alg_name = "CatBoost"

print(f"number of errors for alg {alg_name}: {len(errors_by_alg[alg_name])}")
print(f"number of unique errors: {len(set(errors_by_alg[alg_name]))}")


number of errors for alg CatBoost: 30
number of unique errors: 1


In [13]:
# what are the unique errors?
print(errors_by_alg[alg_name][0])

Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experiment.py", line 137, in __call__
    result = cross_validation(model, self.dataset, self.time_limit)
  File "/home/shared/tabzilla/TabSurvey/tabzilla_utils.py", line 236, in cross_validation
    loss_history, val_loss_history = curr_model.fit(
  File "/home/shared/tabzilla/TabSurvey/models/tree_models.py", line 151, in fit
    self.model.fit(X, y, eval_set=(X_val, y_val))
  File "/opt/conda/envs/gbdt/lib/python3.9/site-packages/catboost/core.py", line 4716, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/opt/conda/envs/gbdt/lib/python3.9/site-packages/catboost/core.py", line 2021, in _fit
    train_params = self._prepare_train_params(
  File "/opt/conda/envs/gbdt/lib/python3.9/site-packages/catboost/core.py", line 1953, in _prepare_train_params
    _check_train_params(params)
  File "_catboos

## VIME

In [14]:
alg_name = "VIME"

print(f"number of errors for alg {alg_name}: {len(errors_by_alg[alg_name])}")
print(f"number of unique errors: {len(set(errors_by_alg[alg_name]))}")


number of errors for alg VIME: 20
number of unique errors: 8


In [15]:
# what are the unique errors?
unique_errors = list(set(errors_by_alg[alg_name]))

In [16]:
for i, e in enumerate(unique_errors):
    print(f"error {i}:\n{e}\n")

error 0:
Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experiment.py", line 137, in __call__
    result = cross_validation(model, self.dataset, self.time_limit)
  File "/home/shared/tabzilla/TabSurvey/tabzilla_utils.py", line 210, in cross_validation
    raise TimeoutException(f"time limit of {time_limit}s reached during fold {i}")
tabzilla_utils.TimeoutException: time limit of 7200s reached during fold 7


error 1:
Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experiment.py", line 137, in __call__
    result = cross_validation(model, self.dataset, self.time_limit)
  File "/home/shared/tabzilla/TabSurvey/tabzilla_utils.py", line 210, in cross_validation
    raise TimeoutException(f"time limit of {time_limit}s reached during fold {i}")
tabzilla_utils.TimeoutException: time limit of 7200s reached during fold 6


error 2:
Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experimen

All VIME errors are due to timeout...

## TabNet

In [17]:
alg_name = "TabNet"

print(f"number of errors for alg {alg_name}: {len(errors_by_alg[alg_name])}")
print(f"number of unique errors: {len(set(errors_by_alg[alg_name]))}")


number of errors for alg TabNet: 21
number of unique errors: 6


In [18]:
# what are the unique errors?
unique_errors = list(set(errors_by_alg[alg_name]))

In [19]:
for i, e in enumerate(unique_errors):
    print(f"error {i}:\n{e}\n")

error 0:
Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experiment.py", line 137, in __call__
    result = cross_validation(model, self.dataset, self.time_limit)
  File "/home/shared/tabzilla/TabSurvey/tabzilla_utils.py", line 210, in cross_validation
    raise TimeoutException(f"time limit of {time_limit}s reached during fold {i}")
tabzilla_utils.TimeoutException: time limit of 7200s reached during fold 6


error 1:
Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experiment.py", line 137, in __call__
    result = cross_validation(model, self.dataset, self.time_limit)
  File "/home/shared/tabzilla/TabSurvey/tabzilla_utils.py", line 210, in cross_validation
    raise TimeoutException(f"time limit of {time_limit}s reached during fold {i}")
tabzilla_utils.TimeoutException: time limit of 7200s reached during fold 8


error 2:
Traceback (most recent call last):
  File "/home/shared/tabzilla/TabSurvey/tabzilla_experimen

All TabNet errors are due to timeout...